In [1]:
# 日線 進場
# c1 日線收在下布林
# c2 本Ｋ為綠Ｋ 前一天為紅Ｋ
# c3 量大於前日
# c4 收盤大於前日開盤 (discarded)
# rsi > 40

# 日線 出場
# tp_rsi = 68
# tp_slowk = 80
# tp_slowd = 80

# 停損
# 進場前一根低點


In [2]:
import websocket
import requests
import json
import pandas as pd
import datetime as dt
import numpy as np
import time
import ta
from dotenv import load_dotenv
import os
from binance.client import Client
from IPython.display import clear_output
import winsound


In [3]:
frequency = 250  # Set Frequency To 2500 Hertz
duration = 300  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [4]:
load_dotenv()

api_key = os.getenv('API_KEY')
api_secret = os.getenv('SECRET_KEY')

client = Client(api_key, api_secret, testnet = False)

spot = client.get_account()
s_bal = pd.DataFrame(spot['balances'])
print(s_bal)


     asset        free      locked
0      BTC  0.00000000  0.00000000
1      LTC  0.00000000  0.00000000
2      ETH  0.00000000  0.00000000
3      NEO  0.00000000  0.00000000
4      BNB  0.24622450  0.00000000
..     ...         ...         ...
525  MAGIC  0.00000000  0.00000000
526   HIFI  0.00000000  0.00000000
527    RPL  0.00000000  0.00000000
528    GFT  0.00000000  0.00000000
529    GNS  0.00000000  0.00000000

[530 rows x 3 columns]


In [5]:
timezone = 8
endpoint = 'wss://stream.binance.com:9443/ws'
symbol = 'ethusdt'
symbol_C = symbol.upper()
interval = '1d'

# start epoch till now
start_time = 1660092207000
end_time = round(time.time() * 1000)

# step between timestamps in milliseconds, 60000 = 1min 
step = 60000 * 36000

In [6]:
data = json.dumps({'method':'SUBSCRIBE','params':[symbol + '@kline_' + interval],'id':1})

In [7]:
# 先抓歷史資料好讓技術指標能成型
def get_historical(symbol, interval, start_time, end_time, step):
    
    rawdf = pd.DataFrame()
    
    url = "https://api.binance.com/api/v3/klines"
    
    for timestamp in range(start_time, end_time, step):
        params = {"symbol": symbol_C,
                  "interval": interval,
                  "startTime": timestamp,
                  "endTime": timestamp + step}
        response = requests.get(url, params=params).json()
        out = pd.DataFrame(response, columns = ["Open time", "Open", "High", "Low", "Close",
                                               "Volume", "Close_Time", "Quote asset volume",
                                               "Number of trades", "Taker buy base asset volume",
                                               "Taker buy quote asset volume", "Ignore"])
        rawdf = pd.concat([rawdf, out], axis = 0)
    
    rawdf = rawdf[['Close_Time', 'Open', 'Close', "High", "Low", 'Volume']]
    convert_dict = {'Close_Time': float, 'Open': float, 'Close': float, "High": float, "Low": float, 'Volume': float}
    rawdf = rawdf.astype(convert_dict)

    rawdf['Close_Time'] = pd.to_datetime(rawdf['Close_Time'], unit = 'ms')
    rawdf['Close_Time'] = rawdf['Close_Time'] + pd.Timedelta(hours=timezone)
    rawdf['Close_Time'] = rawdf['Close_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    rawdf = rawdf.reset_index(drop=True)
    
    return rawdf

In [8]:
rawdf = get_historical(symbol, interval, start_time, end_time, step)
print(rawdf)

              Close_Time     Open    Close     High      Low        Volume
0    2022-08-12 07:59:59  1853.58  1880.19  1942.00  1850.32  1.105262e+06
1    2022-08-13 07:59:59  1880.19  1958.28  1964.71  1853.06  7.176281e+05
2    2022-08-14 07:59:59  1958.28  1983.55  2020.00  1946.50  6.958760e+05
3    2022-08-15 07:59:59  1983.55  1935.31  2030.00  1906.00  5.695835e+05
4    2022-08-16 07:59:59  1935.32  1899.06  2012.47  1872.31  9.491623e+05
..                   ...      ...      ...      ...      ...           ...
188  2023-02-16 07:59:59  1555.70  1674.92  1680.00  1542.55  5.812079e+05
189  2023-02-17 07:59:59  1674.92  1637.84  1742.97  1633.72  8.327237e+05
190  2023-02-18 07:59:59  1637.83  1693.87  1722.00  1630.51  5.799347e+05
191  2023-02-19 07:59:59  1693.81  1691.62  1713.13  1679.77  2.673058e+05
192  2023-02-20 07:59:59  1691.62  1697.43  1709.63  1689.50  7.124947e+04

[193 rows x 6 columns]


In [9]:
def indicators(df):
#     bband
    bb_int = 30
    bb_dev = 2
    bb = ta.volatility.BollingerBands(df['Close'], window=bb_int, window_dev=bb_dev)
    df['bb_u'] = bb.bollinger_hband()
    df['bb_m'] = bb.bollinger_mavg()
    df['bb_l'] = bb.bollinger_lband()
#     rsi
    rsi_int = 14
    df['rsi'] = ta.momentum.RSIIndicator(df['Close'], window = rsi_int).rsi()
# kd
    kd_int = 14
    d_int = 3
   
    kddf = pd.DataFrame()
    kddf[str(kd_int) + '-Low'] = df['Low'].rolling(kd_int).min()
    kddf[str(kd_int) + '-High'] = df['High'].rolling(kd_int).max()
    df['slowk'] = (df['Close'] - kddf[str(kd_int) + '-Low'])*100/(kddf[str(kd_int) + '-High'] - kddf[str(kd_int) + '-Low'])
    df['slowd'] = df['slowk'].rolling(d_int).mean()
    return df

In [10]:
def conditions(df):

    # c1 日線收在下布林
    # c2 本Ｋ為綠Ｋ 前一天為紅Ｋ
    # c3 量大於前日
    # c4 收盤大於前日開盤 (discarded)
    # rsi > 40

    # c1
    df['c1'] = df['Close'] < df['bb_m']
    # c2
    df['c2'] = (df['Close'] - df['Open'] > 0) & (df['Close'].shift(1) - df['Open'].shift(1) < 0) 
    # c3
    df['c3'] = df['Volume'] > df['Volume'].shift(1)
    # c4
    df['c4'] = df['Close'] > df['Open']
    # c5
    df['c5'] = df['rsi'].shift(-1) >= 40

    # 條件達成
    df['signal'] = False
    df.loc[df.c1 & df.c2 & df.c3 & df.c4 & df.c5 , 'signal'] = True
            
    # 下一根進場
    df['openbuy'] = False

    return df

In [11]:
def enter_position(df):
    global symbol_C
    
    #-----Calculate entry price-----#
    close_val = df['Close']
    low_val = df['Low']
    df.loc[df.index[-1], 'entry_p'] = close_val.loc[close_val.index[-2]]
    df.loc[df.index[-1], 'stop_loss'] = low_val.loc[low_val.index[-2]]

    #-----position attributes-----#
    quantity = 0.005  # the amount of the cryptocurrency you want to buy or sell
    entry_p = df.loc[df.index[-2], 'Close']  # the price at which you want to buy or sell
    stop_loss_p = df.loc[df.index[-2], 'Low'] * 0.999  # the stop loss price
#     stop_limit_price = df.loc[df.index[-1], 'stop_loss']  # the limit price of the stop loss order
#     stop_loss_quantity = 0.005  # the amount of the cryptocurrency you want to sell in the stop loss order

    print(df.loc[df.index[-1], 'entry_p'] == entry_p)  
    
    try:
        order = client.futures_create_order(
            symbol=symbol_C,
            side='BUY',
            type='LIMIT',
            timeInForce='GTC',
            quantity=quantity,
            price=entry_p,
            positionSide='LONG'
        )
        print('Order created successfully.')       
        
        stop_loss_order = client.futures_create_order(
            symbol=symbol_C,
            side='SELL',
            type='STOP_MARKET',
            stopPrice=stop_loss_p,
            stopLimitTimeInForce='GTC',
            closePosition = 'true',
            positionSide = 'LONG'
#             quantity=stop_loss_quantity,
#             stopLimitPrice=stop_limit_price,
        )
        print('Stop loss created successfully')
            
    except Exception as e:
        print(f'Error creating order: {e}')

    print(str(symbol_C) + ' entered at ' + str(entry_p) + ', stop loss at ' + str(stop_loss_p))


In [12]:
line_count = 0
max_lines = 5

current_k = 0
    
tp_rsi = 68
tp_slowk = 80
tp_slowd = 80

def on_open(ws):
    ws.send(data)
    
def on_message(ws, message):
    global current_k, tp_rsi, tp_slowk, tp_slowd, line_count, max_lines
    df = get_historical(symbol, interval, start_time, end_time, step)

    indicators(df)
    conditions(df)
    
    try:
        # 如果時間標籤改變，代表上個 Close 發生，檢查是否進場
        if df['Close_Time'][len(df) - 1] != current_k:
            print('time changed')
            current_k = df['Close_Time'][len(df) - 1]
            if df.iloc[df.index[-2]]['signal'] == True:
                winsound.Beep(frequency, duration)
                df.loc[df.index[-1], 'openbuy'] = True
                enter_position(df)
                print(df.tail(2))
        
        # 隨時偵測出場條件是否成立，不必等 Close 發生
        if ((df.loc[df.index[-1], 'rsi'] >= tp_rsi) & (df.loc[df.index[-1], 'slowk'] >= tp_slowk) & (df.loc[df.index[-1], 'slowd'] >= tp_slowd)):
            take_profit_order = client.futures_create_order(
                symbol=symbol_C,
                side='SELL',
                type='TAKE_PROFIT_MARKET',
                stopPrice=df.loc[df.index[-1], 'Close'],
                stopLimitTimeInForce='GTC',
                closePosition = 'true',
                positionSide = 'LONG'
            )
            winsound.Beep(frequency, duration)
            print('Take profit created successfully')
        
    except Exception as e:
        print(f'Error creating order: {e}')


    try:
        df = df.reset_index(drop=True)
        print('nothing happened...' + str(df.loc[df.index[-1], 'Close_Time']) + ' ' + str(df.loc[df.index[-1], 'Close']))
        print()
        print(df.tail(1))
        print('----------------------------------------------------------------------------')
        
        positions = client.futures_account()['positions']
        for position in positions:
            if float(position['positionAmt']) != 0:
                position_df = pd.DataFrame({'Symbol':position['symbol'],
                                            'Entry_P':round(float(position['entryPrice']),2),
                                            'Amt':round(float(position['positionAmt']) * df.loc[df.index[-1], 'Close'],2),
                                            'PL':round(float(position['unRealizedProfit']),2),
                                            'X':round(float(position['leverage']),1),
                                            'Liquid_P':round(float(position['liquidationPrice']),2)
                                           }, index=[0])      
                print(position_df)
    
    except Exception as e:
        print(f'Error: {e}')    
        
        
    line_count += 1
    if line_count >= max_lines:
        clear_output(wait=True)
        line_count = 0

In [13]:
ws = websocket.WebSocketApp(endpoint, on_message = on_message, on_open = on_open)
ws.run_forever()

False